In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from env import get_db_url

### Acquire the data

In [2]:
# getting data
# df = pd.read_clipboard()

In [3]:
# data acquired
df = pd.read_csv('anonymized-curriculum-access.txt', delim_whitespace= True, names=['date', 'time', 'lesson', 'clicks', 'size_', 'ip'])

In [4]:
# check
df.head()

,date,time,lesson,clicks,size_,ip
0,2018-01-26,09:55:03,/,1,8,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22,97.105.19.61


In [5]:
df.size_.value_counts

<bound method IndexOpsMixin.value_counts of 0          8
1          8
2          8
3          8
4         22
          ..
900218    28
900219    28
900220    28
900221    28
900222    28
Name: size_, Length: 900223, dtype: object>

In [6]:
# checking 
df.ip.value_counts()

97.105.19.58       268648
97.105.19.61        60530
192.171.117.210      9124
71.150.217.33        6791
76.185.145.231       4754
                    ...  
172.56.7.179            1
172.58.30.200           1
107.77.217.20           1
71.135.71.73            1
99.43.140.149           1
Name: ip, Length: 5216, dtype: int64

In [7]:
# more info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900223 entries, 0 to 900222
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    900223 non-null  object
 1   time    900223 non-null  object
 2   lesson  900223 non-null  object
 3   clicks  900223 non-null  int64 
 4   size_   899803 non-null  object
 5   ip      847749 non-null  object
dtypes: int64(1), object(5)
memory usage: 41.2+ MB


In [8]:
# first lets adjust the date and time to be one 
df['date'] = pd.to_datetime(df['date'] +' '+ df['time'])

In [9]:
# dropping time
df.drop('time', axis=1, inplace=True)

In [10]:
# now lets inspect the clocks and size becasue I'm not sure hwta they are
# The size appears to have both objects and floats in the series, i think some of the the ip address got into the size column. I'm also gonna change that nbame to something selse

In [11]:
# hmmmm
df[df.size_.isna() == True]

,date,lesson,clicks,size_,ip
10953,2018-02-16 16:38:04,/,103,NaN,64.157.241.246
27257,2018-03-18 11:58:29,/,88,NaN,99.57.17.35
55513,2018-05-12 12:11:27,/,88,NaN,99.57.17.35
55514,2018-05-12 12:11:34,/,88,NaN,99.57.17.35
76480,2018-06-23 15:35:26,/,103,NaN,167.220.152.20
...,...,...,...,...,...
591846,2020-07-08 11:13:42,/,88,NaN,70.94.165.250
591847,2020-07-08 11:13:45,/,88,NaN,70.94.165.250
716654,2020-10-30 11:40:27,/,88,NaN,66.55.32.74
716655,2020-10-30 11:40:42,/,88,NaN,66.55.32.74


In [12]:
# checking in on these nulls
ip_in_size = df[df.ip.isna() == True].size_

In [13]:
# correcting the mistaken ip addresses
df.ip.fillna(ip_in_size, inplace=True)

In [14]:
# now our ip addresses are corrected
df.ip.isna().sum()

0

In [15]:
# these are the nulls from the size_ columns
df.size_.isna().sum()

420

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900223 entries, 0 to 900222
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    900223 non-null  datetime64[ns]
 1   lesson  900223 non-null  object        
 2   clicks  900223 non-null  int64         
 3   size_   899803 non-null  object        
 4   ip      900223 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 34.3+ MB


##### Takeaways:
* The clicks and size column may need to be merged together? I'm not sure what those numbers mean but it may be to do with the lessons
* the only secitons with null values now is the size, but this amount is incorrect. I will inspect and replace the size that fit the ip addresses

In [17]:
ip_in_size.index

Int64Index([   411,    412,    425,    435,    436,    437,    438,    439,
               440,    441,
            ...
            899892, 899893, 899894, 899895, 899896, 899897, 899898, 899899,
            899900, 899902],
           dtype='int64', length=52474)

In [18]:
# replacing the ip address in size with nulls
df['size_'].loc[ip_in_size.index] = '1111'

/var/folders/fj/7fbysvvn7956npcj9d4mxfxw0000gn/T/ipykernel_1774/35126913.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size_'].loc[ip_in_size.index] = '1111'


In [19]:
# im really not sure what these are at all
df.size_.value_counts().sort_index().head()

1        8890
11        253
1111    52474
12        302
13       2845
Name: size_, dtype: int64

In [20]:
# i dont know what these are either tbh,  but there are quite a few more in here than in size
df.clicks.value_counts()

11     17913
64     16347
53     12329
314     7783
1       7404
       ...  
66         1
163        1
918        1
212        1
952        1
Name: clicks, Length: 981, dtype: int64

In [21]:
# lookin at what changes when these two are brough together
(df['clicks'] + df['size_'].dropna().astype(int)).value_counts()

39.0      15178
92.0      12530
67.0       8197
29.0       7590
456.0      6919
          ...  
1078.0        1
1056.0        1
676.0         1
1014.0        1
79.0          1
Length: 874, dtype: int64

In [22]:
# experimenting
df['clicks'] + ('.' + df['size_']).dropna().astype(float)

0           1.80
1           1.80
2           1.80
3           1.80
4           2.22
           ...  
900218     64.28
900219     64.28
900220     64.28
900221    744.28
900222     64.28
Length: 900223, dtype: float64

In [23]:
(df['clicks'] + ('.' + df['size_']).dropna().astype(float)).value_counts()

11.280     15178
64.280     12530
53.140      8197
495.510     6451
428.280     5819
           ...  
592.280        1
879.135        1
619.570        1
952.170        1
652.280        1
Length: 1029, dtype: int64

In [24]:
df.groupby('size_')['clicks'].nunique()

size_
1       18
11       5
1111    77
12       5
13      10
132     23
133     17
134     20
135     31
137     21
138     37
139     26
14      13
15       5
16       8
17      14
18      14
19      17
2        3
21      28
22      35
23      33
24      34
25      26
26      20
27      29
28      51
29      25
31      28
32      22
33      29
34      23
4        1
5        1
51      27
52      28
53      29
55      21
56      24
57      29
58      23
59      29
6        2
61      23
62      33
7        6
8        5
9        1
Name: clicks, dtype: int64

In [25]:
df.groupby('clicks')['size_'].nunique()

clicks
1      2
2      1
3      1
4      1
5      1
      ..
977    1
978    1
979    1
980    1
981    1
Name: size_, Length: 981, dtype: int64

In [26]:
# renaming the columns to match
df = df.rename({'clicks':'user_id',
          'size_':'cohort_id'}, axis=1)

In [27]:
# there are no 0's at all
df.cohort_id.dropna().astype(int).value_counts().sort_index()

1        8890
2          93
4           4
5           1
6          72
7         598
8        1712
9           5
11        253
12        302
13       2845
14       9587
15        691
16        755
17       4954
18       2158
19       1237
21       7444
22      30926
23      28534
24      35636
25      25586
26      27749
27      20743
28      84031
29      38096
31      25359
32      29356
33      40730
34      26538
51      32888
52      28033
53      36902
55      21581
56      33568
57      33844
58      29855
59      32015
61      17713
62      37109
132     23691
133     14715
134     16623
135     16397
137      8562
138      7276
139      1672
1111    52474
Name: cohort_id, dtype: int64

In [28]:
# it appears that the user_ids are correct
df.user_id.dropna().astype(int).value_counts().sort_index().index.to_list()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [29]:
# checkling out the values in here to see if we can find the missing cohorts
df[df.cohort_id.isna()].groupby('ip').count()

,date,lesson,user_id,cohort_id
ip,,,,
107.77.217.130,2,2,2,0
107.77.217.208,4,4,4,0
107.77.217.90,2,2,2,0
107.77.219.65,4,4,4,0
107.77.220.72,1,1,1,0
12.219.109.178,10,10,10,0
130.45.39.198,22,22,22,0
167.220.152.20,31,31,31,0
173.173.127.101,167,167,167,0


In [30]:
# finding the number of unique of each
nunique_ip = df.groupby('ip').nunique()

In [31]:
nunique_ip[nunique_ip.cohort_id == 0]

,date,lesson,user_id,cohort_id
ip,,,,
107.77.217.130,2,2,1,0
107.77.217.208,4,3,1,0
107.77.217.90,2,1,1,0
107.77.219.65,4,3,1,0
107.77.220.72,1,1,1,0
12.219.109.178,7,9,1,0
130.45.39.198,20,8,1,0
167.220.152.20,22,30,1,0
173.224.2.235,3,1,1,0


In [32]:
nunique_ip.user_id.value_counts()

1      4899
2       470
3        98
4        17
5        15
7         6
6         6
17        3
24        2
8         2
10        2
21        1
128       1
402       1
15        1
85        1
30        1
306       1
26        1
11        1
9         1
14        1
Name: user_id, dtype: int64

### Trying to parse the lessons sections

In [33]:
df.lesson.str.split('/')

0                                                      [, ]
1                                                 [java-ii]
2                    [java-ii, object-oriented-programming]
3                     [slides, object_oriented_programming]
4                              [javascript-i, conditionals]
                                ...                        
900218                              [jquery, personal-site]
900219                                 [jquery, mapbox-api]
900220                          [jquery, ajax, weather-map]
900221    [anomaly-detection, discrete-probabilistic-met...
900222                                 [jquery, mapbox-api]
Name: lesson, Length: 900223, dtype: object

In [34]:
# look at lessons
lesson = df.lesson.str.split('/')

In [35]:
def parse_log_entry(parts):
    '''
    
    '''
#     parts = entry.split('/')
    
    
    output = {}
    
    if len(parts) >= 1:
        
        output['topic'] = parts[0]
        
        if len(parts) >= 2:
            
            output['lesson'] = parts[1]
        
        else: 
            
            output['lesson'] = ' '
    
    return pd.Series(output)

In [36]:
lesson.head().apply(parse_log_entry)

,topic,lesson
0,,
1,java-ii,
2,java-ii,object-oriented-programming
3,slides,object_oriented_programming
4,javascript-i,conditionals


In [37]:
# parsing the lesson
parsed_lesson = lesson.apply(parse_log_entry)

KeyboardInterrupt: 

In [ ]:
# adding the parsed topic and lesson together
df = pd.concat([df.drop('lesson', axis=1), parsed_lesson], axis=1)

In [ ]:
# checking
df.head()

In [38]:
### Acquire part 2
query = '''SELECT *
FROM `logs` lg
JOIN cohorts c ON lg.cohort_id = c.id;'''

db = 'curriculum_logs'

df = pd.read_sql(query, get_db_url(db))

In [39]:
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2


In [49]:
# parsing the path for the different topics and lessons
df[['topic', 'lesson']] = df.path.str.split('/').dropna().apply(parse_log_entry)

In [48]:
# checlking the none type
df[df.path.isna() == True]

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,topic,lesson
478630,2020-04-08,09:25:18,None,586,55.0,72.177.240.51,55,Curie,#curie,2020-02-03,2020-07-07,2020-02-03 19:31:51,2020-02-03 19:31:51,None,3,NaN,NaN


In [50]:
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,topic,lesson
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1,,
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1,java-ii,
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1,java-ii,object-oriented-programming
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1,slides,object_oriented_programming
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2,javascript-i,conditionals


### 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?

In [92]:
pd.DataFrame(df[df.program_id == 1].groupby('cohort_id')['topic'].value_counts()[df[df.program_id == 1].groupby('cohort_id')['topic'].value_counts() > 900].sort_values(ascending=False))

topic
cohort_id topic              
13.0      content        2254
17.0      content        1359
1.0       javascript-i   1333
14.0      content        1128
          spring         1104
1.0       html-css       1031
14.0      javascript-i   1006
17.0      javascript-i    953
1.0       spring          914

In [90]:
pd.DataFrame(df[df.program_id == 2].groupby('cohort_id')['topic'].value_counts()[df[df.program_id == 2].groupby('cohort_id')['topic'].value_counts() > 1000].sort_values(ascending=False))

topic
cohort_id topic              
33.0      javascript-i   6994
28.0                     6340
          javascript-i   6301
33.0      html-css       5929
29.0      javascript-i   5860
...                       ...
52.0      appendix       1079
27.0      java-i         1037
58.0      toc            1022
53.0      search         1020
61.0      appendix       1011

[238 rows x 1 columns]

In [89]:
pd.DataFrame(df[df.program_id == 3].groupby('cohort_id')['topic'].value_counts()[df[df.program_id == 3].groupby('cohort_id')['topic'].value_counts() > 1000].sort_values(ascending=False))

topic
cohort_id topic                      
59.0      classification         5030
137.0     fundamentals           3451
34.0      1-fundamentals         3124
59.0                             2980
          sql                    2771
133.0     classification         2715
34.0      6-regression           2573
59.0      python                 2572
55.0      3-sql                  2499
133.0     fundamentals           2469
59.0      1-fundamentals         2411
34.0      4-python               2392
55.0      4-python               2330
34.0      3-sql                  2313
137.0     sql                    2277
59.0      fundamentals           2154
55.0      1-fundamentals         2148
59.0      stats                  2049
34.0                             1967
55.0      6-regression           1957
34.0      7-classification       1945
          5-stats                1922
133.0     sql                    1871
55.0                             1712
59.0      anomaly-detection      1649
          regression             1539
          3-sql                  1352
137.0     python                 1349
34.0      8-clustering           1332
133.0     python                 1275
34.0      appendix               1245
55.0      5-stats                1226
          7-classification       1213
34.0      10-anomaly-detection   1200
59.0      clustering             1178
55.0      appendix               1161
133.0     regression             1140
34.0      9-timeseries           1127
          11-nlp                 1118
59.0      appendix               1116
133.0                            1115
59.0      nlp                    1106
133.0     stats                  1008

In [82]:
pd.DataFrame(df[df.program_id == 4].groupby('cohort_id')['topic'].value_counts()[df[df.program_id == 4].groupby('cohort_id')['topic'].value_counts() > 1000])

,,topic
cohort_id,topic,


In [64]:
# We need to identify the programs each group cohort was in
df.groupby(['program_id', 'cohort_id'])['topic'].value_counts()

program_id  cohort_id  topic                       
1           1.0        javascript-i                    1333
                       html-css                        1031
                       spring                           914
                       mysql                            775
                       java-iii                         696
                                                       ... 
3           137.0      ml-methodologies-drawing.jpg       1
                       sql.mysql-overview                 1
                       where                              1
4           9.0        content                            4
                                                          1
Name: topic, Length: 2183, dtype: int64

In [ ]:
# one way to do it
    # get a lost of all cohorts who look into mysql
    # after it is had, tell pandas to put a 'Data Science' for all cohort that match that
    # within that find all cohorts that use html-class, 

### 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [ ]:
df.groupby(['cohort_id', 'topic']).

### 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

### 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

### 5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?

### 6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?

### 7. Which lessons are least accessed?

### 8. Anything else I should be aware of?